In [1]:
#First, we load the required libraries

from cgitb import text
from dataclasses import replace
from gettext import gettext
from msilib.schema import Class
from operator import contains, countOf
from os import getcwd, link
from pickle import TRUE
from typing import Mapping
from unittest import result
from xml.dom.minidom import Element
from attr import attr, attrs
from pyparsing import line

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import requests
import urllib.request
#import time
from bs4 import BeautifulSoup
import shutil

import pandas as pd

from lxml import etree

In [2]:
#--------------------------------------------------------#
#---Programm command lines for web scraping on Indeed----#
#--------------------------------------------------------#

In [2]:
#here we load the browser and the indeed website
browser=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get("https://ca.indeed.com/")
time.sleep(2)   #this command force the programm to wait 2 seconds to make sure the hole page has been loaded 
                #before executing the next commands

In [3]:
#clear the search boxes (job title and location) on the web page
browser.find_element(By.ID, 'text-input-what').click()
try: browser.find_element(By.XPATH, '//input[@ID="text-input-what"]/following::span').click()
except: pass
browser.find_element(By.ID, 'text-input-where').click()
try: browser.find_element(By.XPATH, '//input[@ID="text-input-where"]/following::span').click()
except: pass

In [4]:
#search according to the user preferences of job and location
search_name = input('What job are you looking for?')    #request for the job title the user wants to search for
browser.find_element(By.ID, 'text-input-what').send_keys(search_name)      #paste the job title to the box

search_place = input('In what location do yo want to look for the job?')       #request for the job location the user wants to search for
browser.find_element(By.ID, 'text-input-where').send_keys(search_place)     #paste the job location to the box

browser.find_element(By.XPATH, '//button[@class="yosegi-InlineWhatWhere-primaryButton"]').click()   #search for the results
time.sleep(1)

try: browser.find_element(By.XPATH, '//*[@id="popover-x"]/button').click()       #close popup window
except: pass

In [9]:
#this function will return the path to a specific page on indeed
def test(n):
    next_path = f'//a[@aria-label="{n}"]'   #path structure is obtained through the website html inspection
    return next_path

#this function return the end of url for each page
def end_url(x):
    if x > 1:
        a = x - 1
        end = '&start=' + f'{a}0'
    else:
        end = ""       #first page has no ending
    return end

# function to extract html document from given url
def getHTML(url):
    # request for HTML document of given url
    response = requests.get(url, headers={"Content-Type":"text"})
    # response will be provided in JSON format
    return response.text

# this function estimates the time necessary for downloading data
def delay(nb):
    if nb/20 < 1:
        delay = f'{int(nb/20)*60} sec'
    elif nb/20 >= 1 and nb/20 < 60:
        delay = f'{int(nb/20)} mns'
    else: delay = f'{int(nb/20/60)} hrs {int(((nb/20/60)-int(nb/20/60))*60)} mns'
    return delay

#this function inform on the occurence of a specific string in the job description section
def skill_test(x):
    result = 'check website'
    try:
        string = sub_soup.find('div', attrs={'id':'jobDescriptionText'}).text
        result_list = []
        for each in x:        
            result_list.append(each in string)  # append True/False for each element in substring
        r = any(result_list) #call any() with boolean results list
        if r == True:
            result = 'yes'
        else:
            result = 'no'
    except: pass
    return result

In [6]:
#estimate the time for dowloading data
nb = int(browser.find_element(By.ID, 'searchCountPages').text.split()[3])
print(f'Data on {nb} jobs will be downloaded. Estimated time is {delay(nb)}.')

Data on 1 jobs will be downloaded. Estimated time is 0 sec.


In [8]:
#next lines allow to navigates through all the pages containing job announcement

i = 0
main_url = browser.current_url
jobtab = pd.DataFrame(columns=['TITLE', 'COMPANY', 'LOCATION', 'T_WORK', 'T_CONTRACT', 'SALARY', 'EXCEL', 'PYTHON', 'R', 'SAS', 'STATA', 'VBA', 'SQL', 'POWER_BI', 'WEBSITE'])

while i == 0:

    main_soup = BeautifulSoup(getHTML(main_url), "html.parser")
    time.sleep(2)  
    for each in main_soup.find_all('a', attrs={'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}):
        sub_url = 'https://ca.indeed.com' + each['href']
        sub_soup = BeautifulSoup(getHTML(sub_url), "html.parser")
        time.sleep(2)
        dom = etree.HTML(str(sub_soup))
        time.sleep(2)
        
        #Commands to report data from web pages to database
        title = ''
        comp_name = ''
        location = ''
        t_work = ''
        t_contract =''
        salary = ''
        website = ''

        try:
            title = sub_soup.find('h1').text     #title of the post
        except: pass

        try:
            comp_name = dom.xpath("//div[@class='jobsearch-InlineCompanyRating icl-u-xs-mt--xs jobsearch-DesktopStickyContainer-companyrating']//a")[0].text    #name of the company
        except: pass

        try:
            location = dom.xpath('//div[@class="icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle"]//div[contains(text(),"QC")]')[0].text.replace(', QC','')
        except: pass
        
        try:
            t_work = dom.xpath('//div[@class="icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle"]//div[contains(text(),"QC")]/../following-sibling::div/div')[0].text
        except: pass

        try:
            t_contract = sub_soup.find('span', attrs={'class':'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs'}).text      #type of contract (full time/part time, casual/permanent)
        except: pass
        
        try:
            salary = sub_soup.find('span', attrs={'class':'icl-u-xs-mr--xs attribute_snippet'}).text      #salary
        except: pass

        excel = skill_test(['Excel', 'excel', 'EXCEL', 'MSExcel', 'MSexcel'])   #check if SAS skill is required
        python = skill_test(['Python', 'python', 'PYTHON'])   #check if SAS skill is required
        r = skill_test(['R,', 'R.', 'R)'])   #check if R skill is required
        sas = skill_test(['SAS'])   #check if SAS skill is required
        stata = skill_test(['Stata', 'stata', 'STATA'])   #check if SAS skill is required
        vba = skill_test(['VBA'])   #check if SAS skill is required
        sql = skill_test(['SQL'])   #check if SQL skill is required
        power_bi = skill_test(['Power BI', 'PowerBI', 'powerBI', 'power BI', 'BI visualization'])       #check if Power BI skill is required

                                                        #set the new line of observation       
        obs = pd.DataFrame([[title, comp_name, location, t_work, t_contract, salary, excel, python, r, sas, stata, vba, sql, power_bi, sub_url]], columns=['TITLE', 'COMPANY', 'LOCATION', 'T_WORK', 'T_CONTRACT', 'SALARY', 'EXCEL', 'PYTHON', 'R', 'SAS', 'STATA', 'VBA', 'SQL', 'POWER_BI', 'WEBSITE'])
        
        jobtab = pd.concat([jobtab, obs], axis=0)       #adding the new observation to the table

    try:
        main_url = browser.find_element(By.XPATH, '//head/link[@rel="next"]').get_attribute('href')
        browser.get(main_url)
    except NoSuchElementException:
        i = 1

print(jobtab)

try:
    deljobs = browser.find_element(By.CLASS_NAME, 'dupetext').text.split()[3]
    print(f'Note: {deljobs} jobs have been removed to avoid duplicated results')
except: pass

Empty DataFrame
Columns: [TITLE, COMPANY, LOCATION, T_WORK, T_CONTRACT, SALARY, EXCEL, PYTHON, R, SAS, STATA, VBA, SQL, POWER_BI, WEBSITE]
Index: []


In [119]:
jobtab.to_csv('draft/jobs.csv', index=False, encoding='UTF-8', sep=';')       #save the database to csv file

In [ ]:
df = pd.DataFrame(['http://google.com', 'http://duckduckgo.com'])

def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, 'val')

df.style.format(make_clickable)

In [ ]:
main_url = browser.current_url
main_soup = BeautifulSoup(getHTML(main_url), "html.parser")
main_soup

In [ ]:
def test_state(x):
    for each in x:
        states = states + 'contains(text(),"{x}")'
    return states

can_states = ["AB", "BC", "PE", "MB", "NB", "NS", "NU", "ON", "QC", "SK", "NL", "NT", "YT"]

test_state(can_states)